In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import math

In [3]:
# set working directory
print(os.getcwd())
folder_path = "../src"

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # Change the current working directory to the specified folder
    os.chdir(folder_path)
    print(f"Current working directory set to: {folder_path}")
else:
    print(f"The folder '{folder_path}' does not exist.")

c:\Users\luisf\Documents\Python Scripts\Cell-Tracking\src
Current working directory set to: ../src


In [4]:
# function to check if pixel is within a contour - is flourescence within a cell

def is_pixel_inside_contour(pixel, contour):
    """
    Checks if a pixel is inside a contour.

    Args:
        pixel: A tuple (x, y) representing the pixel's coordinates.
        contour: A list of contour points, obtained from cv2.findContours.

    Returns:
        True if the pixel is inside the contour, False otherwise.
    """
    x, y = pixel
    point = (x, y)
    return cv2.pointPolygonTest(contour, point, False) >= 0



In [5]:
# # downscale image
# scale_percent = 60 # percent of original size
# width = int(image.shape[1] * scale_percent / 100)
# height = int(image.shape[0] * scale_percent / 100)
# dim = (width, height)

# #

In [7]:
# read the image
#image = cv2.imread('pngs/cells_C0_T178.png')

# function to segment an image with watershedding
# input: image to segment
# output: list of contours (1 for each cell)
def do_watershed(img):
    # convert to grayscale
    grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # threshold image 
    thresh = cv2.threshold(grayscale,0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    
    # get sure background area
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    sure_bg = cv2.dilate(thresh, kernel, iterations=3) 
    
    # Distance transform
    dist = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)

    # get foreground area
    ret, sure_fg = cv2.threshold(dist, 0.2 * dist.max(), 255, cv2.THRESH_BINARY)
    sure_fg = sure_fg.astype(np.uint8) 
    
    # get unknown area
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    # connected components
    ret, markers = cv2.connectedComponents(sure_fg)
    
    # Add one to all labels so that background is not 0, but 1
    markers += 1
    # mark the region of unknown with zero
    markers[unknown == 255] = 0
    
    # apply watershed algorithm
    markers = cv2.watershed(img, markers)
    
    labels = np.unique(markers)

    cells = []
    for label in labels[2:]:
        # Create a binary image in which only the area of the label is in the foreground 
        # and the rest of the image is in the background   
        target = np.where(markers == label, 255, 0).astype(np.uint8)

        # Perform contour extraction on the created binary image
        contours, hierarchy = cv2.findContours(
            target, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        cells.append(contours[0])
    
    return cells
    

In [8]:
# kernel=(3,3)
# blur = cv2.GaussianBlur(grayscale, kernel, 0)
# plt.imshow(blur)
# plt.show()

In [12]:
# #sharpen image
# sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
# sharpen = cv2.filter2D(grayscale, -1, sharpen_kernel)
# plt.imshow(sharpen)
# plt.show()

In [10]:
# function to get center coords for every cell
# input: list of contours
# output: list of tuples, where each tuple is the xy coord of one cell
def get_centers(contours):
    # get COM for each contour from previous step
    coms = []

    for contour in contours:
        # Calculate moments for the contour
        M = cv2.moments(contour)

        # Calculate the centroid (center) of the contour
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = 0, 0  # Handle division by zero if the contour has no area

        # add centroid to list of cell centers
        coms.append((cx,cy))
        #cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1)
    
    return coms

# function to get contours for every frame of a video
# input: list of images
# output: list of lists of contours
def get_all_contours(frames):
    all_frames_contours = []
    for frame in frames:
        contours = do_watershed(frame)
        all_frames_contours.append(contours)
        
    return all_frames_contours

def get_all_centers(frames_contours):
    all_frames_centers = []
    for frame_contours in frames_contours:
        centers = get_centers(frame_contours)
        all_frames_centers.append(centers)
        
    return all_frames_centers

# function to loop over directory and read in all images
def read_images_from_folder(folder_path):
    image_list = []

    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"The folder '{folder_path}' does not exist.")
        return []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files and read images
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)

        # Check if the file is an image (you can add more image extensions if needed)
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tif')):
            image = cv2.imread(file_path)
            
            # Append the image to the list if it was successfully read
            if image is not None:
                image_list.append(image)
    
    return image_list
    
def dist_between_points(coord_a, coord_b):
    x1, y1 = coord_a
    x2, y2 = coord_b
    
    # Calculate the Euclidean distance between the two points
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance


In [11]:
# read in all images
all_frames = read_images_from_folder("pngs")

# get all the contours for all frames
all_contours = get_all_contours(all_frames)
# now get all the centers from this list of lists
all_centers = get_all_centers(all_contours)


In [12]:
# build dictionary of cells
cells_dict = {}
for i in range(0,len(all_centers[0])):
    curr_name = i
    # initialize empty list
    cells_dict[i] = []
    cells_dict[i].append(all_centers[0][i])

In [13]:
# do object tracking
# loop over every frame of the video
for i in range(1, len(all_centers)):
    # loop over all the centers, and for each center find the most likely center in the next frame
    curr_frame_coords = all_centers[i]

    for cell in cells_dict:
        # get most recent coordinate of cell from dictionary
        prev_points = cells_dict[cell]
        prev_point = prev_points[len(prev_points)-1]
        # initialize list of distances
        dists = {}
        for curr_point in curr_frame_coords:
            # get distance from every other point
            dists[curr_point] = dist_between_points(prev_point, curr_point)

        # add coord with smallest distance 
        curr_smallest_dist = float('inf')
        curr_smallest_coord = None
        for point in dists:
            if dists[point] < curr_smallest_dist:
                curr_smallest_dist = dists[point]
                curr_smallest_coord = point

        # add closest coordinate to cell dictionary
        cells_dict[cell].append(curr_smallest_coord)
    


In [14]:
for coord in cells_dict:
    print(cells_dict[coord])

[(510, 511), (510, 511), (510, 511)]
[(87, 4), (86, 4), (85, 4)]
[(945, 1), (945, 1), (946, 2)]
[(926, 7), (925, 7), (925, 8)]
[(941, 11), (940, 10), (940, 11)]
[(684, 24), (687, 22), (690, 21)]
[(920, 28), (918, 29), (917, 29)]
[(111, 38), (112, 37), (115, 37)]
[(907, 44), (909, 43), (908, 43)]
[(976, 41), (976, 42), (976, 44)]
[(60, 45), (56, 51), (57, 49)]
[(58, 50), (56, 51), (57, 49)]
[(789, 53), (788, 49), (788, 42)]
[(770, 53), (772, 58), (773, 56)]
[(772, 60), (772, 58), (773, 56)]
[(755, 67), (754, 65), (753, 66)]
[(1003, 64), (1005, 66), (1003, 71)]
[(931, 66), (929, 66), (931, 65)]
[(927, 71), (927, 71), (927, 70)]
[(642, 73), (639, 73), (636, 74)]
[(680, 79), (680, 80), (681, 78)]
[(761, 80), (761, 79), (760, 90)]
[(648, 86), (647, 86), (647, 85)]
[(20, 91), (20, 90), (21, 90)]
[(612, 92), (610, 90), (607, 89)]
[(674, 93), (668, 93), (663, 94)]
[(779, 94), (778, 89), (779, 87)]
[(741, 97), (739, 100), (738, 103)]
[(752, 103), (752, 105), (738, 103)]
[(765, 101), (765, 100),